In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json

In [2]:
# file1Path = "/Users/JLJXDXCFK7/Prediction/author-test-140-origin.json"
# file1Path = "/Users/JLJXDXCFK7/Prediction/authorpred-challenge.149.recall.json"
file1Path = "/Users/JLJXDXCFK7/Prediction/author-challenge-149-origin.json"
# file2Path = "/Users/JLJXDXCFK7/Prediction/glip-test-148.json"
file2Path = "/Users/JLJXDXCFK7/Prediction/glip-chall-224g.json"
outPath = "/Users/JLJXDXCFK7/Prediction/merge.json"

print("Input file1:",file1Path)
print("Input file2:",file2Path)
print("Output file:",outPath)

with open(file1Path, 'r+') as inFile:
    jsonObj1 = json.load(inFile)
with open(file2Path, 'r+') as inFile:
    jsonObj2 = json.load(inFile)

Input file1: /Users/JLJXDXCFK7/Prediction/author-challenge-149-origin.json
Input file2: /Users/JLJXDXCFK7/Prediction/glip-chall-224g.json
Output file: /Users/JLJXDXCFK7/Prediction/merge.json


In [3]:
new_result = []
classes = [
    "Penalty",
    "Kick-off",
    "Goal",
    "Substitution",
    "Offside",
    "Shots on target",
    "Shots off target",
    "Clearance",
    "Ball out of play",
    "Throw-in",
    "Foul",
    "Indirect free-kick",
    "Direct free-kick",
    "Corner",
    "Yellow card",
    "Red card",
    "Yellow->red card"
]
exclude = []
eitherClazz = list(set([]) - set(exclude))
bothClazz = list(set([]) - set(eitherClazz) - set(exclude))
file2Clazz = list(set(["Red card", "Yellow->red card", "Direct free-kick"]) - set(bothClazz) - set(eitherClazz) - set(exclude))
file1Clazz = list(set(classes) - set(file2Clazz) - set(bothClazz) - set(eitherClazz) - set(exclude))

for clazz in bothClazz:
    assert(clazz in classes)
for clazz in file1Clazz:
    assert(clazz in classes)
for clazz in file2Clazz:
    assert(clazz in classes)
for clazz in eitherClazz:
    assert(clazz in classes)

print("Either class:", eitherClazz)
print("Both class:", bothClazz)
print("File1 class:", file1Clazz)
print("File2 class:", file2Clazz)

map1 = {}
map2 = {}
mapBoth = {}
mapFinal = {}
avgScore1 = {}
avgScore2 = {}
avgScoreBoth = {}
for clazz in classes:
    map1[clazz] = 0
    map2[clazz] = 0
    mapBoth[clazz] = 0
    mapFinal[clazz] = 0
    avgScore1[clazz] = []
    avgScore2[clazz] = []
    avgScoreBoth[clazz] = []

for games1, games2 in zip(jsonObj1, jsonObj2):
    assert(games1['video']==games2['video'])

    events1 = games1['events']
    events2 = games2['events']
    video = games1['video']
    fps = games1['fps']
    new_events = []

    map_frame = {}

    for event in events1:
        label = event['label']
        frame = event['frame']
        score = event['score']
        if (frame not in map_frame):
            map_frame[frame] = []
        map_frame[frame].append(label)
        if (label in eitherClazz):
            new_events.append(event)
        if (label in file1Clazz):
            new_events.append(event)
        avgScore1[label].append(score)
        map1[label] += 1

    for event in events2:
        label = event['label']
        frame = event['frame']
        score = event['score']
        if ((frame in map_frame) and (label in map_frame[frame])):
            mapBoth[label] += 1
            avgScoreBoth[label].append(score)
            if (label in bothClazz):
                new_events.append(event)
        if (label in file2Clazz):
            new_events.append(event)
        if (label in eitherClazz):
            new_events.append(event)
        map2[label] += 1
        avgScore2[label].append(score)

    new_game = {
        'video': video,
        'events': new_events,
        'fps': fps
    }
    for event in new_events:
        label = event['label']
        mapFinal[label] += 1
    new_result.append(new_game)
print(len(new_result))
print("File 1:", map1)
print("File 2:", map2)
print("Join both:", mapBoth)
print("Result:", mapFinal)
for clazz in classes:
    avgScore1[clazz] = sum(avgScore1[clazz])/len(avgScore1[clazz])
    avgScore2[clazz] = sum(avgScore2[clazz])/len(avgScore2[clazz])
    avgScoreBoth[clazz] = sum(avgScoreBoth[clazz])/len(avgScoreBoth[clazz])
print(avgScore1)
print(avgScore2)
print(avgScoreBoth)

Either class: []
Both class: []
File1 class: ['Indirect free-kick', 'Penalty', 'Kick-off', 'Ball out of play', 'Offside', 'Foul', 'Goal', 'Corner', 'Substitution', 'Shots off target', 'Shots on target', 'Throw-in', 'Clearance', 'Yellow card']
File2 class: ['Yellow->red card', 'Direct free-kick', 'Red card']
100
File 1: {'Penalty': 33, 'Kick-off': 6929, 'Goal': 1525, 'Substitution': 8299, 'Offside': 4553, 'Shots on target': 6081, 'Shots off target': 5655, 'Clearance': 16984, 'Ball out of play': 31899, 'Throw-in': 24527, 'Foul': 18476, 'Indirect free-kick': 24186, 'Direct free-kick': 3387, 'Corner': 3140, 'Yellow card': 4827, 'Red card': 82, 'Yellow->red card': 75}
File 2: {'Penalty': 189541, 'Kick-off': 593530, 'Goal': 592230, 'Substitution': 593575, 'Offside': 593575, 'Shots on target': 593575, 'Shots off target': 593575, 'Clearance': 593575, 'Ball out of play': 593575, 'Throw-in': 593575, 'Foul': 593575, 'Indirect free-kick': 593575, 'Direct free-kick': 593259, 'Corner': 593269, 'Yell

In [4]:
with open(outPath, 'w+') as outFile:
    json.dump(new_result, outFile)